
Congrats....again! Based off the amazing work you did on the classification model, you've been promoted to chief of all data scientists in Hollywood, your a total celebrity....among data scientists :).  

Your boss, head of the studio, has now asked you to build a model to predict gross revenue in order to help them decide which movies to invest in.

Once again, you would like to be able to explain the model to mere mortals but need a fairly robust and flexible approach so you've chosen to use decision trees to get started. 

In doing so...you leverage work you've done in the past to get the job done....you're a data scientist after all! 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import graphviz
from sklearn.model_selection import train_test_split,GridSearchCV,RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay, get_scorer_names
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import plot_tree, DecisionTreeClassifier, export_graphviz
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import random
from numpy import log

In [4]:
#1. Load the data
#Sometimes need to set the working directory back out of a folder that we create a file in

#import os
#os.listdir()
#print(os.getcwd())
#os.chdir('c:\\Users\\Brian Wright\\Documents\\3001Python\\DS-3001')

movie=pd.read_csv("../data/movie_metadata.csv")



#2 Ensure all the variables are classified correctly and collapse factor variables as needed.

In [5]:
movie.describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
count,4993.000000,5028.000000,4939.000000,5020.000000,5036.000000,4.159000e+03,5.043000e+03,5043.000000,5030.000000,5022.000000,4.551000e+03,4935.000000,5030.000000,5043.000000,4714.000000,5043.000000
mean,140.194272,107.201074,686.509212,645.009761,6560.047061,4.846841e+07,8.366816e+04,9699.063851,1.371173,272.770808,3.975262e+07,2002.470517,1651.754473,6.442138,2.220403,7525.964505
std,121.601675,25.197441,2813.328607,1665.041728,15020.759120,6.845299e+07,1.384853e+05,18163.799124,2.013576,377.982886,2.061149e+08,12.474599,4042.438863,1.125116,1.385113,19320.445110
min,1.000000,7.000000,0.000000,0.000000,0.000000,1.620000e+02,5.000000e+00,0.000000,0.000000,1.000000,2.180000e+02,1916.000000,0.000000,1.600000,1.180000,0.000000
25%,50.000000,93.000000,7.000000,133.000000,614.000000,5.340988e+06,8.593500e+03,1411.000000,0.000000,65.000000,6.000000e+06,1999.000000,281.000000,5.800000,1.850000,0.000000
50%,110.000000,103.000000,49.000000,371.500000,988.000000,2.551750e+07,3.435900e+04,3090.000000,1.000000,156.000000,2.000000e+07,2005.000000,595.000000,6.600000,2.350000,166.000000
75%,195.000000,118.000000,194.500000,636.000000,11000.000000,6.230944e+07,9.630900e+04,13756.500000,2.000000,326.000000,4.500000e+07,2011.000000,918.000000,7.200000,2.350000,3000.000000
max,813.000000,511.000000,23000.000000,23000.000000,640000.000000,7.605058e+08,1.689764e+06,656730.000000,43.000000,5060.000000,1.221550e+10,2016.000000,137000.000000,9.500000,16.000000,349000.000000


#3 Check for missing variables and correct as needed.

#4 Guess what, you don't need to scale the data, because DTs don't require this to be done, they make local greedy decisions...keeps getting easier, go to the next step.

#5 Determine the variance of the target variable, is it normally distributed? If not, you may want to transform the target variable. Why might this be important?

#6 Split your data into test, tune, and train. (80/10/10)

In [9]:
np.random.seed(1)
#Steps 2-6 are done in the following function:

# Function drops unnecessary columns, fills in missing vals with mean, creates the score column with the binned imdb values, and creates the train/test/tune splits
def clean_split_data(df, target, train_size=0.80, tune_size=0.10, random_state=21):
    # Split independent and dependent variables
    df = df[['num_critic_for_reviews','actor_1_facebook_likes','gross','num_voted_users','cast_total_facebook_likes','num_user_for_reviews','budget','imdb_score','movie_facebook_likes']]
    df['score'] = pd.cut(df['imdb_score'], bins=[0, 5, 7, 10], labels=['low', 'medium', 'high'])
    df = df.drop(columns=['imdb_score'])
    df = df.fillna(df.mean(numeric_only=True))
    X = df.drop(columns=target)
    y = df[target]
    
    # Calculate prevalence of the score
    variance = y.value_counts().var()
    print("Variance of Imdb scores groups by range:", variance)
    
    # Split data into training and testing sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=train_size, stratify=y, random_state=random_state)
    
    # Split the temporary set into tuning and testing sets
    X_tune, X_test, y_tune, y_test = train_test_split(X_temp, y_temp, train_size=tune_size, stratify=y_temp, random_state=random_state+28)
    
    return X_train, X_tune, X_test, y_train, y_tune, y_test

# Use the function to split the movie data
X_train, X_tune, X_test, y_train, y_tune, y_test = clean_split_data(movie, 'score')

Variance of Imdb scores groups by range: 1446877.0


/tmp/ipykernel_1455/3343801637.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = pd.cut(df['imdb_score'], bins=[0, 5, 7, 10], labels=['low', 'medium', 'high'])


#7 Create the kfold object for cross validation.

In [7]:
# k fold object creation
kf = RepeatedStratifiedKFold(n_splits=10, n_repeats =5, random_state=42)

#8 Create the scoring metric (several measures) you will use to evaluate your model and choose a hyperparamater to optimize.

In [8]:
#setting scoring metrics and param for grid search
scoring = ['accuracy', 'recall_macro', 'balanced_accuracy', 'f1_macro']

param={"max_depth" : [1,2,3,4,5,6,7,8,9,10],
           }

#9 Build the regression tree object. 

#10 Fit the model to the training data.

#11 Explore the results of the model. Print the output of the grid search. What is the best parameter and score?

#12 Which variables appear to be contributing the most (variable importance) 

#13 Create a model object using the best model hyperparameter value from the trained regression tree. 

#14 Using the best model predict on the tune data and print out the results.

#15 How does the model perform on the tune data as compared to the training data?

#16 What five movies are predicted to have the lowest gross revenue from the tune set? 

#17 As a data scientist you are always looking to improve your results. What are some ideas you have to improve the model? Have you ask...WHAT IF?!

#18 Experiment using your what if ideas (two is fine). Did the model improve? If so, how much? If not, what did you learn? (You don't need to include all the "what if" code just speak to what you tried and what you learned).

#19 Use the best model to predict on the test data and print out the evaluation results.

#20 Does the model perform well on the test data when compared to the training and tune data? Why or why not? 

#21 Where are the errors occurring in your predictions? Are they systematic or random? In either case, what are the implications of this?

#22 Summarize what you learned along the way and make recommendations on how this could be used moving forward, being careful not to over promise.